In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import math

In [5]:
tickers = ["NVDA", "NFLX", "AMC", "RIVN", "PLTR", "AMZN", "TSLA"]
etfs = ["SPY", "IWM", "DIA"]

class DataDownloader:
    def __init__(self, tickers, etfs):
        self.tickers = tickers
        self.etfs = etfs

    def download(self, period='1y'):
        return yf.download(self.tickers + self.etfs, period=period)['Adj Close']

    def quarterly(self):
        return self.download(period='3mo')

    def annual(self):
        return self.download(period='1y')

# Usage:
downloader = DataDownloader(tickers, etfs)
quarterly_data = downloader.quarterly()
annual_data = downloader.annual()

[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed


In [13]:
quarterly_data.tail()


,AMC,AMZN,DIA,IWM,NFLX,NVDA,PLTR,RIVN,SPY,TSLA
Date,,,,,,,,,,
2023-09-11,7.3400,143.100006,347.369995,184.360001,445.359985,451.779999,15.790,23.410000,448.450012,273.579987
2023-09-12,7.5700,141.229996,347.209991,184.179993,434.690002,448.700012,15.590,23.580000,445.989990,267.480011
2023-09-13,8.2400,144.850006,346.549988,182.970001,412.239990,454.850006,15.600,23.250000,446.510010,271.299988
2023-09-14,8.1400,144.720001,349.940002,185.550003,400.489990,455.809998,15.830,24.110001,450.359985,276.040009
2023-09-15,8.3789,140.884995,347.119995,183.419998,395.695007,442.369995,15.345,24.100000,444.760010,273.926605


In [12]:
annual_data.tail()

,AMC,AMZN,DIA,IWM,NFLX,NVDA,PLTR,RIVN,SPY,TSLA
Date,,,,,,,,,,
2023-09-11,7.3400,143.100006,347.369995,184.360001,445.359985,451.779999,15.790,23.410000,448.450012,273.579987
2023-09-12,7.5700,141.229996,347.209991,184.179993,434.690002,448.700012,15.590,23.580000,445.989990,267.480011
2023-09-13,8.2400,144.850006,346.549988,182.970001,412.239990,454.850006,15.600,23.250000,446.510010,271.299988
2023-09-14,8.1400,144.720001,349.940002,185.550003,400.489990,455.809998,15.830,24.110001,450.359985,276.040009
2023-09-15,8.3789,140.895004,347.119995,183.419998,395.695007,442.325012,15.345,24.100000,444.760010,273.929993


# Part 1: We'll create an index with our Tickers:
           


In [16]:
table = pd.DataFrame(index=tickers)
table

""
NVDA
NFLX
AMC
RIVN
PLTR
AMZN
TSLA


# Part 2: We'll calculate weight using the following formula:
           
###   Price / Total Price

In [18]:
# Taking the last row of quartely_data.tail(), for most current price. Then summed to calculate portfolio value...
# assuming that each stock is bought only 1 time.
current_prices = quarterly_data[tickers].iloc[-1]
summed_prices = current_price.sum()


#Each value in the row 'current_price' will be calculated to append it's weight % to a new column 
weight_list = []
for price in current_prices:
    weight = (price/summed_prices) * 100
    weight_list.append(round(weight,2))
    
table['Weight'] = weight_list
table

,Weight
NVDA,34.01
NFLX,30.42
AMC,0.64
RIVN,1.85
PLTR,1.18
AMZN,10.83
TSLA,21.06


# Part 3: We'll calculate 3-month trailing Annualized Volatility using the following formula:
           
### ( STD [ Daily Return ] )( SQRT [ Trading Days] ) * 100

In [20]:
def calculate_annualized_volatility(ticker_data):
    daily_return = ticker_data.pct_change().dropna()
    std_dev = daily_return.std()
    trading_days = daily_return.count()
    annualized_volatility = std_dev * math.sqrt(trading_days) * 100
    return annualized_volatility 


volatilities = {}
for ticker in tickers:
    volatilities[ticker] = calculate_annualized_volatility(quarterly_data[ticker])
    
#Create Annualized votality column
for stock, volatility in volatilities.items():    
    table.loc[stock, 'Annualized Volatility'] = volatility

In [21]:
table

,Weight,Annualized Volatility
NVDA,34.01,19.992491
NFLX,30.42,17.913787
AMC,0.64,85.624624
RIVN,1.85,36.437004
PLTR,1.18,32.519068
AMZN,10.83,15.230778
TSLA,21.06,26.953440


In [22]:
annual_data

,AMC,AMZN,DIA,IWM,NFLX,NVDA,PLTR,RIVN,SPY,TSLA
Date,,,,,,,,,,
2022-09-15,98.800003,126.279999,304.250092,178.709274,235.380005,129.210953,8.010,39.689999,383.851807,303.750000
2022-09-16,89.800003,123.529999,302.788544,176.111740,240.130005,131.899323,7.780,39.259998,380.923431,303.350006
2022-09-19,91.800003,124.660004,304.852631,177.617126,243.630005,133.738205,7.860,37.939999,383.877441,309.070007
2022-09-20,87.099998,122.190002,301.776093,175.196686,242.850006,131.679459,7.620,35.750000,379.471100,308.730011
2022-09-21,86.000000,118.540001,296.655090,172.520416,236.869995,132.528946,7.690,35.099998,372.851654,300.799988
...,...,...,...,...,...,...,...,...,...,...
2023-09-11,7.340000,143.100006,347.369995,184.360001,445.359985,451.779999,15.790,23.410000,448.450012,273.579987
2023-09-12,7.570000,141.229996,347.209991,184.179993,434.690002,448.700012,15.590,23.580000,445.989990,267.480011
2023-09-13,8.240000,144.850006,346.549988,182.970001,412.239990,454.850006,15.600,23.250000,446.510010,271.299988


# Part 4-6: 12-month trailing beta agaisnt's etfs:
           


In [26]:
annual_data

,AMC,AMZN,DIA,IWM,NFLX,NVDA,PLTR,RIVN,SPY,TSLA
Date,,,,,,,,,,
2022-09-15,98.800003,126.279999,304.250092,178.709274,235.380005,129.210953,8.010,39.689999,383.851807,303.750000
2022-09-16,89.800003,123.529999,302.788544,176.111740,240.130005,131.899323,7.780,39.259998,380.923431,303.350006
2022-09-19,91.800003,124.660004,304.852631,177.617126,243.630005,133.738205,7.860,37.939999,383.877441,309.070007
2022-09-20,87.099998,122.190002,301.776093,175.196686,242.850006,131.679459,7.620,35.750000,379.471100,308.730011
2022-09-21,86.000000,118.540001,296.655090,172.520416,236.869995,132.528946,7.690,35.099998,372.851654,300.799988
...,...,...,...,...,...,...,...,...,...,...
2023-09-11,7.340000,143.100006,347.369995,184.360001,445.359985,451.779999,15.790,23.410000,448.450012,273.579987
2023-09-12,7.570000,141.229996,347.209991,184.179993,434.690002,448.700012,15.590,23.580000,445.989990,267.480011
2023-09-13,8.240000,144.850006,346.549988,182.970001,412.239990,454.850006,15.600,23.250000,446.510010,271.299988


In [27]:
# First define daily returns
returns = annual_data.pct_change().dropna()

# Function to calculate beta
def calculate_beta(stock, etf):
    covariance = returns[stock].cov(returns[etf])
    variance = returns[etf].var()
    beta = covariance / variance
    return beta

# Calculate beta for each stock against each ETF and append to table
for etf in etfs:
    beta_column = []
    for stock in tickers:
        beta = calculate_beta(stock, etf)
        beta_column.append(beta)
    table[f'Beta against {etf}'] = beta_column




In [28]:
table

,Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA
NVDA,34.01,19.992491,2.066239,1.281925,1.795280
NFLX,30.42,17.913787,1.448125,0.985022,1.426000
AMC,0.64,85.624624,1.816113,1.700954,1.835619
RIVN,1.85,36.437004,2.334803,1.854304,2.226877
PLTR,1.18,32.519068,1.853571,1.452279,1.623241
AMZN,10.83,15.230778,1.500549,0.983311,1.317384
TSLA,21.06,26.953440,1.772572,1.300460,1.442572
